In [10]:
import napari
import numpy as np
import skimage.io
from scipy.ndimage import gaussian_filter, convolve
from skimage import data, color, filters, transform
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.measure import label
from scipy import ndimage
import os
import io


In [11]:
def apply_3d_sobel_filter(image, smoothpixelsize):
    # Create the 3D Sobel filters
    sobel3Dz = np.array([[[1, 2, 1], [2, 4, 2], [1, 2, 1]], 
                        [[0, 0, 0], [0, 0, 0], [0, 0, 0]], 
                        [[-1, -2, -1], [-2, -4, -2], [-1, -2, -1]]], dtype=float)
    
    sobel3Dx = np.transpose(sobel3Dz, (1, 2, 0))
    sobel3Dy = np.transpose(sobel3Dz, (2, 0, 1))
    
    


    # Prepare output variable
    XYZedges = []

    image = np.double(image)

        # Smooth the individual images
    if smoothpixelsize is not None:
        if isinstance(smoothpixelsize, (int, float)):  # Check if smoothpixelsize is a scalar
            if smoothpixelsize > 0:
                image = gaussian_filter(image, sigma=smoothpixelsize)
        elif isinstance(smoothpixelsize, (list, tuple, np.ndarray)) and len(smoothpixelsize) == 3:
            if all(s > 0 for s in smoothpixelsize):  # Check if all values are positive
                image = gaussian_filter(image, sigma=smoothpixelsize)

        # Convolve in the 3 directions
    xedges = convolve(image, sobel3Dx, mode='constant')
    yedges = convolve(image, sobel3Dy, mode='constant')
    zedges = convolve(image, sobel3Dz, mode='constant')

        # Take the sum of squares of the 3 volumetric images
    xyzedges = np.sqrt(xedges**2 + yedges**2 + zedges**2)

        # Make the outer edge zero
    xyzedges[[0, -1], :, :] = 0
    xyzedges[:, :, [0, -1]] = 0
    xyzedges[:, [0, -1], :] = 0

        # Store output
    XYZedges.append(xyzedges)

    return XYZedges

In [12]:
# Config

#Voxel size in µm
vsz = 0.1579
vsy = 0.1245
vsx = 0.1245
target_voxelsize = 0.1245 #microns

voxelsize = np.asarray([vsz, vsx, vsy])
scaling_factors = voxelsize / target_voxelsize

In [13]:
# create viewer
viewer = napari.Viewer(ndisplay=3)

18-Oct-23 11:05:44 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x679-71+17 (frame: 1104x726-80-21) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1360x851-70+24 (frame: 1378x898-79-14) margins: 9, 38, 9, 9 minimum size: 374x575 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=392,622 maxtrack=0,0)


In [14]:
num_droplet = 9

DATA_ROOT = os.path.join("C:\\","Users","Heloise","Documents","Data","Naked1","%s" %num_droplet)
image_ini = skimage.io.imread(os.path.join(DATA_ROOT, 'droplet%s.tif' %num_droplet))
Dec = skimage.io.imread(os.path.join(DATA_ROOT, 'Dec10_median.tif' ))
#Dec = skimage.io.imread(os.path.join(DATA_ROOT, 'contour.tif' ))
#Dec2 = skimage.io.imread(os.path.join(DATA_ROOT, 'Dec_61.tif'))
n_frames = Dec.shape[0]
viewer.add_image(image_ini, name='Raw data', scale = voxelsize)
viewer.add_image(Dec, name='deconvoluted_data1', scale = voxelsize)
#viewer.add_image(Dec2, name='deconvoluted_data2', scale = voxelsize)
#viewer.add_image(image_ini,  name='Raw data')
#viewer.add_image(Dec, name='deconvoluted_data')

<Image layer 'deconvoluted_data1' at 0x1532fbf2df0>

In [15]:
edge = apply_3d_sobel_filter(Dec, smoothpixelsize = [1,1,2])  #Resonant [1,1,2]
viewer.add_image(edge, scale = voxelsize)

<Image layer 'edge' at 0x1533f0d6520>

In [16]:
name_file = 'edges.tif'
saving_path = DATA_ROOT.replace("\\", "/")

skimage.io.imsave('%s/%s' %(saving_path, name_file), np.array(edge).astype(np.float32))  